In [1]:
import numpy as np
import pandas as pd
import requests,json,os
from urllib.parse import quote 
import os,glob

In [2]:
df = pd.read_excel('data/서울시 관광 자연_ljh.XLSX', engine='openpyxl')

In [3]:
df.to_csv('data/main/temp_files/서울시 관광 자연_ljh.csv', index=False)
df = pd.read_csv('data/main/temp_files/서울시 관광 자연_ljh.csv', encoding='utf-8', sep=',')

In [4]:
df = df[(df['언어']) == 'ko']
# 도로명주소만 남기기
df.rename(columns= {'상호명':'관광지명', '신주소':'도로명주소'}, inplace=True)
df = df[['관광지명', '도로명주소']]
df.reset_index(drop=True,inplace=True)
df.nunique()

관광지명     120
도로명주소    110
dtype: int64

In [5]:
df.to_csv('data/서울_자연.csv',index=False)
df= pd.read_csv('data/서울_자연.csv', encoding='utf-8', sep=',')

In [6]:
df.isna().sum().sum()

0

In [18]:
# 도로명주소에서 우편번호 제거
addr_list = []
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i]= ' '.join(addr1)
    addr_list.append(df.도로명주소[i])

In [19]:
# 도로명주소에서 우편번호 제거
for i in df.index:
    addr1 = df.도로명주소[i].split(' ')[1:]
    df.도로명주소[i] = (' '.join(addr1))

In [20]:
# 2 kakao API로 도로명 주소 구하는 함수
def kakao_location(place):
    with open('data/main/key/kakaoapikey.txt') as f_:
        kakao_key = f_.read()
    base_url = "https://dapi.kakao.com/v2/local/search/address.json"
    url = f'{base_url}?query={quote(place)}'
    header = {'Authorization':f'KakaoAK {kakao_key}'}
    result = requests.get(url, headers=header).json()
    lat_ = float(result['documents'][0]['y'])
    lng_ = float(result['documents'][0]['x'])
    return lat_,lng_

In [23]:
# 2
temp1 =[]
for i in df.index:
    try:
        temp1.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

7 뚝섬유원지
27 수락산
28 도봉산
35 서리풀 공원
36 찬바람이 쌩쌩 부는 겨울,  찜질방 데이트
46 관악산
47 화랑로 낙엽거리
52 등잔 밑이 어둡다, 우리 동네 보석 같은 공원
54 한강예술공원
69 매봉산 야경
73 소월길
75 안산
78 중랑천 제방
80 홍제천
81 양재천&탄천
85 서울 둘레길 코스 안내
100 노원우주학교
107 매화근린공원 
111 불광천
117 망우산 사색의 길


In [33]:
df.head(118).tail(70)

,관광지명,도로명주소
48,경의선 숲길,연남동
49,서울식물원,"마곡동로 161 (마곡동, 서울식물원)"
50,사직공원,사직로 89 (사직동)
51,백범광장공원,퇴계로8길 49-4
52,"등잔 밑이 어둡다, 우리 동네 보석 같은 공원",
...,...,...
113,불암산,덕릉로94길 73 (중계동)
114,서서울호수공원,"남부순환로64길 20 (신월동, 공원관리사무소)"
115,여의도샛강생태공원,여의동로 48 (여의도샛강생태공원 방문자센터)
116,석촌호수 야경,삼학사로 136 (잠실동)


In [30]:
# 2
temp2 =[]
# for i in df.index:
for i in 100:
    try:
        temp2.append(kakao_location(df.도로명주소.values[i]))
    except:
        print(i, df.관광지명[i])
        
df_test = pd.DataFrame(temp1,columns=('lat','lng'))
df2 = pd.concat([df, df_test], axis=1)

TypeError: 'int' object is not iterable

In [17]:
kakao_location('서울 강서구 공항대로42길 51-22 (내발산동, 강서청소년회관')

(37.5560223205345, 126.843879936706)

In [26]:
df2.lat.isna().sum()

20

In [35]:
df2.tail(23)

,관광지명,도로명주소,lat,lng
97,선유도 공원,"선유로 343 (당산동, 선유도공원)",37.509063,127.097868
98,구암공원,"허준로5길 42 (가양동, 구암공원관리사무실)",37.534831,126.987028
99,감로천 생태공원,"독산로54길 188 (독산동, 금천구민문화체육센터)",37.601382,126.961955
100,노원우주학교,동일로 동일로 205길 13 노원우주학교,NaN,NaN
101,길동생태공원,천호대로 1291 (길동),NaN,NaN
102,서울대공원,"대공원광장로 102 (막계동, 서울대공원)",NaN,NaN
103,봉산,서오릉로23길 8-5 (수국사),NaN,NaN
104,개화산,"양천로 22 (방화동, 개화산역) 개화산",NaN,NaN
105,석촌호수공원,삼학사로 136 (서울놀이마당),NaN,NaN
106,우장공원,"공항대로42길 51-22 (내발산동, 강서청소년회관)",NaN,NaN


In [36]:
df3 = df2[df2['lat'].notnull()]
df3.tail()

,관광지명,도로명주소,lat,lng
95,월드컵공원 반려견놀이터,"증산로 32 (상암동, 안내소)",37.527432,126.830087
96,남산예장공원,퇴계로26길 36 (예장동),37.518718,126.921949
97,선유도 공원,"선유로 343 (당산동, 선유도공원)",37.509063,127.097868
98,구암공원,"허준로5길 42 (가양동, 구암공원관리사무실)",37.534831,126.987028
99,감로천 생태공원,"독산로54길 188 (독산동, 금천구민문화체육센터)",37.601382,126.961955


In [37]:
# 4 처리 끝난 파일 저장
df3.to_csv('data/서울_자연_final.csv',index=False)
# 체크
df4 = pd.read_csv('data/서울_자연_final.csv', encoding='utf-8', sep=',')
df4.head()

,관광지명,도로명주소,lat,lng
0,월드컵공원,마포구 하늘공원로 86,37.571598,126.881674
1,양재시민의 숲,매헌로 99 (양재동),37.473050,127.036977
2,여의도 공원,"여의공원로 68 (여의도동, 여의도공원관리사무소)",37.526809,126.922284
3,몽촌토성,올림픽로 424 (방이동),37.520340,127.115518
4,남산공원,삼일대로 231,37.555633,126.992218
